# Liu & Mei (1994)'s shallow water Bingham model with CentPy in 1D

### Import packages

In [ ]:
# Install the centpy package
!pip install centpy

In [1]:
# Import numpy and centpy for the solution 
import numpy as np
import centpy

In [2]:
# Imports functions from matplotlib and setup for the animation
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [3]:
import os
import csv

### Equation

We solve the Liu & Mei (1994)'s shallow water Bingham equations in 1D

\begin{equation} 
\displaystyle
\partial_t 
\begin{bmatrix} h \\ q \\ u_p \end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} q \\ (M+\displaystyle\frac{h^2}{2\beta}) \\ (\displaystyle\frac{u_p^2}{2}+\displaystyle\frac{h}{\beta}) \end{bmatrix} 
= \begin{bmatrix} 0 \\ \displaystyle\frac{1}{\beta}(h-\beta{\rm {sgn}}(u_p)-\displaystyle\frac{2u_p}{h_o}) \\ \displaystyle\frac{1}{\beta} (1-\alpha\displaystyle\frac{{\rm {sgn}}(u_p)}{h-h_o}) \end{bmatrix}  
\end{equation}
where 
\begin{equation} 
M = u_p^2(h-\frac{7}{15}h_o)
\end{equation}

\begin{equation} 
q = u_p(h-\frac{1}{3}h_o) 
\end{equation}


BC: Periodic box.

Normal flow:
\begin{equation}
h=1,\;h_o=1-\alpha,\;u_p=\frac{1}{2}(1-\alpha)^2
\end{equation}

In [4]:
# problem-specific params
# included in the package in an ugly way, to be fixed in a more elegant way
# but this part should also be kept for BC and initialization
alpha_coeff = 0.30
beta_coeff = 27.0
wave_number = 1.20
L_x = 2.0*np.pi/wave_number
dist_amp = 0.125

In [15]:
pars = centpy.Pars1d(x_init=0.0, x_final=L_x, t_final=100.0, dt_out=1, J=600, cfl=0.25, scheme="fd2")

In [16]:
# swe
class lm1d(centpy.Equation1d):
    def initial_data(self):
        x = self.x
        u = np.zeros((self.J + 4, 3))
        # midpoint = int(self.J / 2) + 2

#         left_v = [1, 0, 1.0 / (self.gamma - 1.0)]
#         right_v = [0.125, 0.0, 0.1 / (self.gamma - 1.0)]
        # normal_flow = [ normal_depth, normal_depth*normal_velocity ]
        u[:,0] = 1.0*(1.0+dist_amp*np.sin(2.0*np.pi*x/L_x))
        u[:,1] = 0.50*((1-alpha_coeff)**2)*(2.0/3.0+alpha_coeff/3.0)
        u[:,2] = 0.50*((1-alpha_coeff)**2)
        # u = normal_flow
        return u

    def boundary_conditions(self, u, t):
        u[0,0] = u[-4,0]
        u[0,1] = u[-4,1]
        u[0,2] = u[-4,2]
        u[1,0] = u[-3,0]
        u[1,1] = u[-3,1]
        u[1,2] = u[-3,2]
        u[-2,0] = u[2,0]
        u[-2,1] = u[2,1]
        u[-2,2] = u[2,2]
        u[-1,0] = u[3,0]
        u[-1,1] = u[3,1]
        u[-1,2] = u[3,2]


    def flux_x(self, u):
        f = np.zeros_like(u)
#         ho = 3.0*(u[:, 0]-u[:, 1]/u[:, 2])
#         M_coeff = (u[:, 0]-7.0/15.0*ho)*((u[:, 2])**2)
        M_coeff = ((u[:, 2])**2)*((-2.0/5.0)*u[:, 0]+(7.0/5.0)*u[:, 1]/u[:, 2])

        f[:, 0] = u[:, 1]
        f[:, 1] = (M_coeff+(u[:, 0]**2)/(2.0*beta_coeff))
        f[:, 2] = (u[:, 2]**2)/2.0+u[:, 0]/beta_coeff

        return f

    def spectral_radius_x(self, u):
        #q = u[:, 1] 
        #vel = q/u[:, 0]
#         return 1.0 * np.abs(vel)
        return 1.25 * np.abs(u[:, 2])

        # return 1.0*np.abs(u_x)  + 1.0*np.sqrt(self.gamma * p / rho)
        
    def source(self, u, t):
        s = np.zeros_like(u)
        q = u[:, 1] 
        vel = q/u[:, 0]
        s[:, 0] = 0.0
        s[:, 1] = gravity_coeff*channel_slope*u[:,0]-fc/2.0*(vel**2)
        

### Solution

In [17]:
eqn = lm1d(pars)
soln = centpy.Solver1dLM(eqn)
soln.solve()

KeyboardInterrupt: 

### Animation

In [11]:
# Animation 

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(16,8))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(1,2,2)

# Set the labels
ax1.set_xlabel('x')
ax1.set_ylabel(r'$h$')
ax1.set_xlabel('x')
ax2.set_ylabel(r'$q$')


# Axis limits and lines
line_u=[]
for ax in [ax1, ax2]:
  ax.set_xlim(0.0, L_x)
  ax.set_ylim(0.0, 2.10)
  line_u.append(ax.plot([], [], linewidth=1, color='b', marker='o', markersize=2)[0])

plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
plt.rcParams.update({'font.family': 'Times New Roman','font.size':19})

# animation function.  This is called sequentially
j0 = slice(2,-2)
def animate(i):
    h = soln.u_n[i,j0,0]
    q = soln.u_n[i,j0,1]
    line_u[0].set_data(soln.x[j0], h)
    line_u[1].set_data(soln.x[j0], q)

plt.close()
anim = animation.FuncAnimation(fig, animate, frames=soln.Nt, interval=100, blit=False);
HTML(anim.to_html5_video())


In [50]:
np.shape(soln.u_n)

(101, 604, 3)

In [51]:
# text files output
un_shape = np.shape(soln.u_n)
frames = un_shape[0]
j0 = slice(2,-2)
for i in range(0, frames):
    t = 0.25*i
    format_string_time = f"{t:.1f}"
    file_name = 'outXYZ_%s' % format_string_time
    with open(file_name, 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(zip(np.transpose(soln.x[j0]),np.transpose(soln.u_n[i,j0,0])))